In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas import DataFrame, concat
from google.colab import drive
drive.mount('/content/drive')
from datetime import date
import time
import statsmodels.api as sm
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
from tqdm import tnrange, tqdm_notebook
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.regression.quantile_regression import QuantReg
import os

import math
from math import radians
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!pip install catboost

In [68]:
from catboost import CatBoostRegressor

In [69]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 3                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 3
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_left = ['Time', 'WS', 'DHI','DNI','RH','T','TARGET', 'Day']

n_features = len(what_to_left)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [70]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/sample_submission.csv')
submission.shape

(7776, 10)

In [71]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset['Date'] = dataset['Day']%365
  dataset.drop('Day', axis=1, inplace=True)
  
  temp = list()
  for i in range(0, len(dataset), 48):
    temp += [24-(list(dataset.DHI[i:i+48]).count(0)*0.5)]*48 

  dataset["SH"] = temp

  return dataset

def cos_time(dataset):
  dataset['sin_time'] = np.sin(2*np.pi*dataset.Time/24)
  dataset['cos_time'] = np.cos(2*np.pi*dataset.Time/24)
  return dataset

In [72]:
def get_yday(when):
  HP = date(2020,1,1)
  results = (when-HP).days
  return results

In [73]:
def 절기24(Date):
  target = 0
  for i in list24:
    if Date < i:
      target = list24.index(i) - 1
      break
  if Date < 5:
    target = 23
  return target

In [74]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def train_to_supervised(train, target, n_in):

    clmns = list(train.columns)

    # 기타 칼럼은 전과 같이 들어갈 것.
    scaled_lst = clmns

    scaled_df = train[scaled_lst]
    target_df = target

    # 미래 몇 번째 항목을 가져올 것인가
    future = [48, 96]

    ### 만약에 스케일링을 하고 싶다면 ###
    # scaled_df 데이터 프레임만 스케일링 하고, 절기랑 TARGET 데이터는 그냥 두면 된다.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    n_vars2 = 1 if type(target_df) is list else target_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # 48과 96 후의 타겟 데이터 2개 붙이기.
    # forecast sequence (t, t+1, ... t+n)
    for i in future:
        cols.append(target_df.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [75]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def test_to_supervised(train, n_in):

    clmns = list(train.columns)

    # 타켓 칼럼의 이름을 여기에 입력
    target = ['TARGET']

    # 클래스 변수로 전환(encoding)할 칼럼을 여기에 입력
#    class_lst = ['season']

    # 기타 칼럼은 전과 같이 들어갈 것.
#    scaled_lst = list(set(clmns) - set(class_lst))

    scaled_df = train.copy()
#    class_df = train[class_lst]

    ### 만약에 스케일링을 하고 싶다면 ###
    # testset의 스케일링은 구조가 상당히 까다로우므로....... 일단 나중에 하기로 함.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # class라 encoding 필요한 절기 끝에 하나만 붙여놓기
#    cols.append(class_df)
#    names += class_lst
    
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [76]:
소한 = get_yday(date(2020,1,6))
대한 = get_yday(date(2020,1,20))
입춘 = get_yday(date(2020,2,4))
우수 = get_yday(date(2020,2,19))
경칩 = get_yday(date(2020,3,6))
춘분 = get_yday(date(2020,3,21))
청명 = get_yday(date(2020,4,5))
곡우 = get_yday(date(2020,4,20))
입하 = get_yday(date(2020,5,6))
소만 = get_yday(date(2020,5,21))
망종 = get_yday(date(2020,6,6))
하지 = get_yday(date(2020,6,22))
소서 = get_yday(date(2020,7,7))
대서 = get_yday(date(2020,7,23))
입추 = get_yday(date(2020,8,8))
처서 = get_yday(date(2020,8,23))
백로 = get_yday(date(2020,9,8))
추분 = get_yday(date(2020,9,23))
한로 = get_yday(date(2020,10,8))
상강 = get_yday(date(2020,10,24))
입동 = get_yday(date(2020,11,8))
소설 = get_yday(date(2020,11,22))
대설 = get_yday(date(2020,12,7))
동지 = get_yday(date(2020,12,22))

list24 = [소한, 대한, 입춘, 우수, 경칩, 춘분, 청명, 곡우, 입하, 소만, 망종, 하지, 소서, 대서, 입추, 처서, 백로, 추분, 한로, 상강, 입동, 소설, 대설, 동지]

In [77]:
m = 60

일출 = [7+47/m, 7+44/m, 7+34/m, 7+18/m, 6+56/m, 6+35/m, 6+12/m, 5+51/m, 5+33/m, 5+19/m, 5+11/m, 5+11/m, 5+17/m, 5+27/m, 5+41/m, 5+54/m, 6+8/m, 6+20/m, 6+33/m, 6+47/m, 7+3/m, 7+20/m, 7+33/m, 7+45/m]
일몰 = [17+28/m, 17+42/m, 17+58/m, 18+15/m, 18+30/m, 18+44/m, 18+58/m, 19+12/m, 19+25/m, 19+39/m, 19+50/m, 19+57/m, 19+56/m, 19+50/m, 19+35/m, 19+15/m, 18+51/m, 18+28/m, 18+6/m, 17+45/m, 17+28/m, 17+17/m, 17+13/m, 17+17/m]
경사각 = [32.92, 36.83, 40.75, 44.67, 48.58, 52.5, 56.42, 60.33, 64.25, 68.16, 72.01, 76, 72.1, 68.16, 64.25, 60.33, 56.42, 52.5, 48.58, 44.67, 40.75, 36.83, 32.92, 29]
남중 = [12+36/m, 12+41/m, 12+44/m, 12+44/m, 12+42/m, 12+38/m, 12+34/m, 12+29/m, 12+27/m, 12+27/m, 12+29/m, 12+32/m, 12+35/m, 12+37/m, 12+36/m, 12+33/m, 12+29/m, 12+23/m, 12+18/m, 12+15/m, 12+14/m, 12+16/m, 12+22/m, 12+29/m]
print(len(일출), len(일몰), len(경사각), len(남중))

24 24 24 24


In [78]:
def HRA(DHI, DNI, season, hour):
  # 위도(latitude) 기준을 일단 임의로 대전으로 설정 (위도 36.19~36.2도)
  latitude = radians(36.2)
  season = int(season)
  # 절기별 대한민국의 경사각
  tilt = radians(경사각[season])

  # 절기별 대한민국 대전의 태양 남중시각
  hra = radians(15*(hour - 남중[season]))

  # 구하려는 알파
  elevation = np.arcsin(np.sin(tilt) * np.sin(latitude) + np.cos(tilt) * np.cos(latitude) * np.cos(hra))

  # 천정각(Zenith Angle)은 90 - 알파
  zenith = radians(90) - elevation

  # GHI는 DHI + DNI * cos(천정각)
  ghi = DHI + DNI *np.cos(zenith)

  return ghi

In [79]:
def encoding(data):
  season_list = [i for i in range(0,24)]
  for k in season_list:
    data['Season_' +f'{k}'] = data['season'] == k
  return data*1

In [80]:
def not_minus(dataset):
  for i in range(0, len(dataset.index)):
    for j in range(0, len(dataset.columns)):
      K = dataset.iloc[i,j]
      if K < 0:
        dataset.iloc[i,j] = 0
  return dataset

In [81]:
dataset = raw_data.copy()
dataset2 = drop_clms(dataset)
dataset2.drop(['Hour', 'Minute'], axis =1, inplace=True)
dataset2

,DHI,DNI,WS,RH,T,TARGET,Time,Date,SH
0,0,0,1.5,69.08,-12,0.0,0.0,0,9.0
1,0,0,1.5,69.06,-12,0.0,0.5,0,9.0
2,0,0,1.6,71.78,-12,0.0,1.0,0,9.0
3,0,0,1.6,71.75,-12,0.0,1.5,0,9.0
4,0,0,1.6,75.20,-12,0.0,2.0,0,9.0
...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,21.5,364,9.0
52556,0,0,2.4,66.79,-4,0.0,22.0,364,9.0
52557,0,0,2.2,66.78,-4,0.0,22.5,364,9.0
52558,0,0,2.1,67.72,-4,0.0,23.0,364,9.0


In [82]:
dataset2['season'] = dataset2.apply(lambda x: 절기24(x['Date']), axis = 1)
dataset3 = cos_time(dataset2)
dataset4 = dataset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
# dataset5 = dataset4.iloc[n_days:,:-1]
# goals = pd.DataFrame(dataset4.Goal, index = dataset4.index)
dataset4

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time
0,0,0,1.5,69.08,-12,0.0,9.0,0.000000,1.000000,23,0.0
1,0,0,1.5,69.06,-12,0.0,9.0,0.130526,0.991445,23,0.5
2,0,0,1.6,71.78,-12,0.0,9.0,0.258819,0.965926,23,1.0
3,0,0,1.6,71.75,-12,0.0,9.0,0.382683,0.923880,23,1.5
4,0,0,1.6,75.20,-12,0.0,9.0,0.500000,0.866025,23,2.0
...,...,...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,9.0,-0.608761,0.793353,0,21.5
52556,0,0,2.4,66.79,-4,0.0,9.0,-0.500000,0.866025,0,22.0
52557,0,0,2.2,66.78,-4,0.0,9.0,-0.382683,0.923880,0,22.5
52558,0,0,2.1,67.72,-4,0.0,9.0,-0.258819,0.965926,0,23.0


In [83]:
dataset5= dataset4.copy()

dataset5['GHI'] = dataset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [84]:
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,1.6,88.11,-11,0.0,9.0,0.258819,0.965926,23,1.0,0.0


In [85]:
#dataset6 = dataset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'season', 'TARGET']]
dataset6 = dataset5[['RH', 'T', 'SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']]
dataset6.head(10)

,RH,T,SH,sin_time,cos_time,GHI,TARGET
0,69.08,-12,9.0,0.000000,1.000000,0.0,0.0
1,69.06,-12,9.0,0.130526,0.991445,0.0,0.0
2,71.78,-12,9.0,0.258819,0.965926,0.0,0.0
3,71.75,-12,9.0,0.382683,0.923880,0.0,0.0
4,75.20,-12,9.0,0.500000,0.866025,0.0,0.0
5,69.29,-11,9.0,0.608761,0.793353,0.0,0.0
6,72.56,-11,9.0,0.707107,0.707107,0.0,0.0
7,72.55,-11,9.0,0.793353,0.608761,0.0,0.0
8,74.62,-11,9.0,0.866025,0.500000,0.0,0.0
9,74.61,-11,9.0,0.923880,0.382683,0.0,0.0


In [86]:
scaler = MinMaxScaler()
# scaler = StandardScaler()
scaler.fit(dataset6)
temp_X = pd.DataFrame(scaler.transform(dataset6), columns = dataset6.columns)
temp_y = pd.DataFrame(dataset6['TARGET'])

## dataset6 가지고 동일하게 to_supervised 함수를 써서 나누면 됩니다.
### 나머지 변수 합치기, 인코딩은 유진이가 해줄 것...

In [87]:
dataset7 = train_to_supervised(temp_X,temp_y, n_days)
dataset7

,RH(t-144),T(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHI(t-144),TARGET(t-144),RH(t-143),T(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHI(t-143),TARGET(t-143),RH(t-142),T(t-142),SH(t-142),sin_time(t-142),cos_time(t-142),GHI(t-142),TARGET(t-142),RH(t-141),T(t-141),SH(t-141),sin_time(t-141),cos_time(t-141),GHI(t-141),TARGET(t-141),RH(t-140),T(t-140),SH(t-140),sin_time(t-140),cos_time(t-140),GHI(t-140),TARGET(t-140),RH(t-139),T(t-139),SH(t-139),sin_time(t-139),cos_time(t-139),...,cos_time(t-6),GHI(t-6),TARGET(t-6),RH(t-5),T(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1),TARGET1(t+48),TARGET1(t+96)
144,0.665404,0.129630,0.083333,0.500000,1.000000,0.0,0.0,0.665188,0.129630,0.083333,0.565263,0.995722,0.0,0.0,0.694622,0.129630,0.083333,0.629410,0.982963,0.0,0.0,0.694297,0.129630,0.083333,0.691342,0.961940,0.0,0.0,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,...,0.853553,0.0,0.0,0.814847,0.129630,0.083333,0.195619,0.896677,0.0,0.0,0.773726,0.111111,0.083333,0.250000,0.933013,0.0,0.0,0.773618,0.111111,0.083333,0.308658,0.961940,0.0,0.0,0.795368,0.111111,0.083333,0.370590,0.982963,0.0,0.0,0.795260,0.111111,0.083333,0.434737,0.995722,0.0,0.0,0.0,0.0
145,0.665188,0.129630,0.083333,0.565263,0.995722,0.0,0.0,0.694622,0.129630,0.083333,0.629410,0.982963,0.0,0.0,0.694297,0.129630,0.083333,0.691342,0.961940,0.0,0.0,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,0.0,0.0,0.703062,0.148148,0.083333,0.853553,0.853553,...,0.896677,0.0,0.0,0.773726,0.111111,0.083333,0.250000,0.933013,0.0,0.0,0.773618,0.111111,0.083333,0.308658,0.961940,0.0,0.0,0.795368,0.111111,0.083333,0.370590,0.982963,0.0,0.0,0.795260,0.111111,0.083333,0.434737,0.995722,0.0,0.0,0.759333,0.092593,0.083333,0.500000,1.000000,0.0,0.0,0.0,0.0
146,0.694622,0.129630,0.083333,0.629410,0.982963,0.0,0.0,0.694297,0.129630,0.083333,0.691342,0.961940,0.0,0.0,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,0.0,0.0,0.703062,0.148148,0.083333,0.853553,0.853553,0.0,0.0,0.702954,0.148148,0.083333,0.896677,0.804381,...,0.933013,0.0,0.0,0.773618,0.111111,0.083333,0.308658,0.961940,0.0,0.0,0.795368,0.111111,0.083333,0.370590,0.982963,0.0,0.0,0.795260,0.111111,0.083333,0.434737,0.995722,0.0,0.0,0.759333,0.092593,0.083333,0.500000,1.000000,0.0,0.0,0.759442,0.111111,0.083333,0.565263,0.995722,0.0,0.0,0.0,0.0
147,0.694297,0.129630,0.083333,0.691342,0.961940,0.0,0.0,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,0.0,0.0,0.703062,0.148148,0.083333,0.853553,0.853553,0.0,0.0,0.702954,0.148148,0.083333,0.896677,0.804381,0.0,0.0,0.725354,0.148148,0.083333,0.933013,0.750000,...,0.961940,0.0,0.0,0.795368,0.111111,0.083333,0.370590,0.982963,0.0,0.0,0.795260,0.111111,0.083333,0.434737,0.995722,0.0,0.0,0.759333,0.092593,0.083333,0.500000,1.000000,0.0,0.0,0.759442,0.111111,0.083333,0.565263,0.995722,0.0,0.0,0.758576,0.111111,0.083333,0.629410,0.982963,0.0,0.0,0.0,0.0
148,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,0.0,0.0,0.703062,0.148148,0.083333,0.853553,0.853553,0.0,0.0,0.702954,0.148148,0.083333,0.896677,0.804381,0.0,0.0,0.725354,0.148148,0.083333,0.933013,0.750000,0.0,0.0,0.725246,0.148148,0.083333,0.961940,0.691342,...,0.982963,0.0,0.0,0.795260,0.111111,0.083333,0.434737,0.995722,0.0,0.0,0.759333,0.092593,0.083333,0.500000,1.000000,0.0,0.0,0.759442,0.111111,0.083333,0.565263,0.995722,0.0,0.0,0.758576,0.111111,0.083333,0.629410,0.982963,0.0,0.0,0.692349,0.111111,0.083333,0.691342,0.961940,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,.

In [88]:
X = dataset7.iloc[:,:-2]
#X = encoding(X)
#X.drop('season', axis =1, inplace=True)

y_1 = pd.DataFrame(dataset7.iloc[:,-2])
y_2 = pd.DataFrame(dataset7.iloc[:,-1])
X

,RH(t-144),T(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHI(t-144),TARGET(t-144),RH(t-143),T(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHI(t-143),TARGET(t-143),RH(t-142),T(t-142),SH(t-142),sin_time(t-142),cos_time(t-142),GHI(t-142),TARGET(t-142),RH(t-141),T(t-141),SH(t-141),sin_time(t-141),cos_time(t-141),GHI(t-141),TARGET(t-141),RH(t-140),T(t-140),SH(t-140),sin_time(t-140),cos_time(t-140),GHI(t-140),TARGET(t-140),RH(t-139),T(t-139),SH(t-139),sin_time(t-139),cos_time(t-139),...,SH(t-6),sin_time(t-6),cos_time(t-6),GHI(t-6),TARGET(t-6),RH(t-5),T(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1)
144,0.665404,0.129630,0.083333,0.500000,1.000000,0.0,0.0,0.665188,0.129630,0.083333,0.565263,0.995722,0.0,0.0,0.694622,0.129630,0.083333,0.629410,0.982963,0.0,0.0,0.694297,0.129630,0.083333,0.691342,0.961940,0.0,0.0,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,...,0.083333,0.146447,0.853553,0.0,0.0,0.814847,0.129630,0.083333,0.195619,0.896677,0.0,0.0,0.773726,0.111111,0.083333,0.250000,0.933013,0.0,0.0,0.773618,0.111111,0.083333,0.308658,0.961940,0.0,0.0,0.795368,0.111111,0.083333,0.370590,0.982963,0.0,0.0,0.795260,0.111111,0.083333,0.434737,0.995722,0.0,0.0
145,0.665188,0.129630,0.083333,0.565263,0.995722,0.0,0.0,0.694622,0.129630,0.083333,0.629410,0.982963,0.0,0.0,0.694297,0.129630,0.083333,0.691342,0.961940,0.0,0.0,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,0.0,0.0,0.703062,0.148148,0.083333,0.853553,0.853553,...,0.083333,0.195619,0.896677,0.0,0.0,0.773726,0.111111,0.083333,0.250000,0.933013,0.0,0.0,0.773618,0.111111,0.083333,0.308658,0.961940,0.0,0.0,0.795368,0.111111,0.083333,0.370590,0.982963,0.0,0.0,0.795260,0.111111,0.083333,0.434737,0.995722,0.0,0.0,0.759333,0.092593,0.083333,0.500000,1.000000,0.0,0.0
146,0.694622,0.129630,0.083333,0.629410,0.982963,0.0,0.0,0.694297,0.129630,0.083333,0.691342,0.961940,0.0,0.0,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,0.0,0.0,0.703062,0.148148,0.083333,0.853553,0.853553,0.0,0.0,0.702954,0.148148,0.083333,0.896677,0.804381,...,0.083333,0.250000,0.933013,0.0,0.0,0.773618,0.111111,0.083333,0.308658,0.961940,0.0,0.0,0.795368,0.111111,0.083333,0.370590,0.982963,0.0,0.0,0.795260,0.111111,0.083333,0.434737,0.995722,0.0,0.0,0.759333,0.092593,0.083333,0.500000,1.000000,0.0,0.0,0.759442,0.111111,0.083333,0.565263,0.995722,0.0,0.0
147,0.694297,0.129630,0.083333,0.691342,0.961940,0.0,0.0,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,0.0,0.0,0.703062,0.148148,0.083333,0.853553,0.853553,0.0,0.0,0.702954,0.148148,0.083333,0.896677,0.804381,0.0,0.0,0.725354,0.148148,0.083333,0.933013,0.750000,...,0.083333,0.308658,0.961940,0.0,0.0,0.795368,0.111111,0.083333,0.370590,0.982963,0.0,0.0,0.795260,0.111111,0.083333,0.434737,0.995722,0.0,0.0,0.759333,0.092593,0.083333,0.500000,1.000000,0.0,0.0,0.759442,0.111111,0.083333,0.565263,0.995722,0.0,0.0,0.758576,0.111111,0.083333,0.629410,0.982963,0.0,0.0
148,0.731631,0.129630,0.083333,0.750000,0.933013,0.0,0.0,0.667677,0.148148,0.083333,0.804381,0.896677,0.0,0.0,0.703062,0.148148,0.083333,0.853553,0.853553,0.0,0.0,0.702954,0.148148,0.083333,0.896677,0.804381,0.0,0.0,0.725354,0.148148,0.083333,0.933013,0.750000,0.0,0.0,0.725246,0.148148,0.083333,0.961940,0.691342,...,0.083333,0.370590,0.982963,0.0,0.0,0.795260,0.111111,0.083333,0.434737,0.995722,0.0,0.0,0.759333,0.092593,0.083333,0.500000,1.000000,0.0,0.0,0.759442,0.111111,0.083333,0.565263,0.995722,0.0,0.0,0.758576,0.111111,0.083333,0.629410,0.982963,0.0,0.0,0.692349,0.111111,0.083333,0.691342,0.961940,0.0,0

# testset은 이미 24절기(season)가 들어있으므로, 약간 다르게 전처리를 해 줘야 함.
## (season) 구하는 과정이 생략됨.
## 이하는 testset을 구하는 코드

In [89]:
file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(1) + '.csv'
new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(1) + '.csv'
file_name = str(1) + '.csv'

# adj_test에서 가져와야 하므로 new_path에서 불러온다
temp = pd.read_csv(new_path)
temp

,Unnamed: 0,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,Time,24
0,0,0,0,0,0,0,2.8,70.34,-6.1,0.0,0.0,22
1,1,0,0,30,0,0,2.9,71.97,-6.4,0.0,0.5,22
2,2,0,1,0,0,0,2.9,69.77,-6.7,0.0,1.0,22
3,3,0,1,30,0,0,2.9,71.95,-7.1,0.0,1.5,22
4,4,0,2,0,0,0,2.9,69.48,-7.4,0.0,2.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...
331,331,6,21,30,0,0,1.7,43.65,-5.2,0.0,21.5,22
332,332,6,22,0,0,0,1.5,40.62,-4.9,0.0,22.0,22
333,333,6,22,30,0,0,1.5,40.62,-4.9,0.0,22.5,22
334,334,6,23,0,0,0,1.5,39.21,-4.9,0.0,23.0,22


In [90]:
temp.drop('Unnamed: 0', axis = 1, inplace = True)

In [91]:
testset = temp.copy()

# 24로 저장한 24절기 season으로 이름 바꾸기
testset.rename(columns = {'24' : 'season'}, inplace = True)


# 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
testset2 = drop_clms(testset)
testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
testset3 = cos_time(testset2)
testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
testset4

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time
0,0,0,2.8,70.34,-6.1,0.0,9.5,0.000000,1.000000,22,0.0
1,0,0,2.9,71.97,-6.4,0.0,9.5,0.130526,0.991445,22,0.5
2,0,0,2.9,69.77,-6.7,0.0,9.5,0.258819,0.965926,22,1.0
3,0,0,2.9,71.95,-7.1,0.0,9.5,0.382683,0.923880,22,1.5
4,0,0,2.9,69.48,-7.4,0.0,9.5,0.500000,0.866025,22,2.0
...,...,...,...,...,...,...,...,...,...,...,...
331,0,0,1.7,43.65,-5.2,0.0,9.5,-0.608761,0.793353,22,21.5
332,0,0,1.5,40.62,-4.9,0.0,9.5,-0.500000,0.866025,22,22.0
333,0,0,1.5,40.62,-4.9,0.0,9.5,-0.382683,0.923880,22,22.5
334,0,0,1.5,39.21,-4.9,0.0,9.5,-0.258819,0.965926,22,23.0


In [92]:
testset5 = testset4.copy()
testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [93]:
testset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,2.1,61.33,-10.8,0.0,9.5,0.258819,0.965926,22,1.0,0.0


In [94]:
#### testset과 dataset이 같은 모양인지 중간점검
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,1.6,88.11,-11,0.0,9.0,0.258819,0.965926,23,1.0,0.0


In [95]:
testset6 = testset5[['RH', 'T', 'SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']]
testset6.head(10)

,RH,T,SH,sin_time,cos_time,GHI,TARGET
0,70.34,-6.1,9.5,0.000000,1.000000,0.0,0.0
1,71.97,-6.4,9.5,0.130526,0.991445,0.0,0.0
2,69.77,-6.7,9.5,0.258819,0.965926,0.0,0.0
3,71.95,-7.1,9.5,0.382683,0.923880,0.0,0.0
4,69.48,-7.4,9.5,0.500000,0.866025,0.0,0.0
5,72.13,-7.9,9.5,0.608761,0.793353,0.0,0.0
6,69.76,-8.3,9.5,0.707107,0.707107,0.0,0.0
7,71.97,-8.7,9.5,0.793353,0.608761,0.0,0.0
8,68.56,-9.1,9.5,0.866025,0.500000,0.0,0.0
9,70.75,-9.5,9.5,0.923880,0.382683,0.0,0.0


In [96]:
testset6 = pd.DataFrame(scaler.transform(testset6), columns = testset6.columns)

In [97]:
testset7 = test_to_supervised(testset6, n_days)
testset7

,RH(t-144),T(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHI(t-144),TARGET(t-144),RH(t-143),T(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHI(t-143),TARGET(t-143),RH(t-142),T(t-142),SH(t-142),sin_time(t-142),cos_time(t-142),GHI(t-142),TARGET(t-142),RH(t-141),T(t-141),SH(t-141),sin_time(t-141),cos_time(t-141),GHI(t-141),TARGET(t-141),RH(t-140),T(t-140),SH(t-140),sin_time(t-140),cos_time(t-140),GHI(t-140),TARGET(t-140),RH(t-139),T(t-139),SH(t-139),sin_time(t-139),cos_time(t-139),...,SH(t-6),sin_time(t-6),cos_time(t-6),GHI(t-6),TARGET(t-6),RH(t-5),T(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1)
144,0.679039,0.238889,0.166667,0.500000,1.000000,0.0,0.0,0.696678,0.233333,0.166667,0.565263,0.995722,0.0,0.0,0.672871,0.227778,0.166667,0.629410,0.982963,0.0,0.0,0.696461,0.220370,0.166667,0.691342,0.961940,0.0,0.0,0.669733,0.214815,0.166667,0.750000,0.933013,0.0,0.0,0.698409,0.205556,0.166667,0.804381,0.896677,...,0.166667,0.146447,0.853553,0.0,0.0,0.454172,0.268519,0.166667,0.195619,0.896677,0.0,0.0,0.481333,0.268519,0.166667,0.250000,0.933013,0.0,0.0,0.480684,0.268519,0.166667,0.308658,0.961940,0.0,0.0,0.499188,0.270370,0.166667,0.370590,0.982963,0.0,0.0,0.494752,0.272222,0.166667,0.434737,0.995722,0.0,0.0
145,0.696678,0.233333,0.166667,0.565263,0.995722,0.0,0.0,0.672871,0.227778,0.166667,0.629410,0.982963,0.0,0.0,0.696461,0.220370,0.166667,0.691342,0.961940,0.0,0.0,0.669733,0.214815,0.166667,0.750000,0.933013,0.0,0.0,0.698409,0.205556,0.166667,0.804381,0.896677,0.0,0.0,0.672763,0.198148,0.166667,0.853553,0.853553,...,0.166667,0.195619,0.896677,0.0,0.0,0.481333,0.268519,0.166667,0.250000,0.933013,0.0,0.0,0.480684,0.268519,0.166667,0.308658,0.961940,0.0,0.0,0.499188,0.270370,0.166667,0.370590,0.982963,0.0,0.0,0.494752,0.272222,0.166667,0.434737,0.995722,0.0,0.0,0.512282,0.274074,0.166667,0.500000,1.000000,0.0,0.0
146,0.672871,0.227778,0.166667,0.629410,0.982963,0.0,0.0,0.696461,0.220370,0.166667,0.691342,0.961940,0.0,0.0,0.669733,0.214815,0.166667,0.750000,0.933013,0.0,0.0,0.698409,0.205556,0.166667,0.804381,0.896677,0.0,0.0,0.672763,0.198148,0.166667,0.853553,0.853553,0.0,0.0,0.696678,0.190741,0.166667,0.896677,0.804381,...,0.166667,0.250000,0.933013,0.0,0.0,0.480684,0.268519,0.166667,0.308658,0.961940,0.0,0.0,0.499188,0.270370,0.166667,0.370590,0.982963,0.0,0.0,0.494752,0.272222,0.166667,0.434737,0.995722,0.0,0.0,0.512282,0.274074,0.166667,0.500000,1.000000,0.0,0.0,0.507845,0.275926,0.166667,0.565263,0.995722,0.0,0.0
147,0.696461,0.220370,0.166667,0.691342,0.961940,0.0,0.0,0.669733,0.214815,0.166667,0.750000,0.933013,0.0,0.0,0.698409,0.205556,0.166667,0.804381,0.896677,0.0,0.0,0.672763,0.198148,0.166667,0.853553,0.853553,0.0,0.0,0.696678,0.190741,0.166667,0.896677,0.804381,0.0,0.0,0.659777,0.183333,0.166667,0.933013,0.750000,...,0.166667,0.308658,0.961940,0.0,0.0,0.499188,0.270370,0.166667,0.370590,0.982963,0.0,0.0,0.494752,0.272222,0.166667,0.434737,0.995722,0.0,0.0,0.512282,0.274074,0.166667,0.500000,1.000000,0.0,0.0,0.507845,0.275926,0.166667,0.565263,0.995722,0.0,0.0,0.532626,0.277778,0.166667,0.629410,0.982963,0.0,0.0
148,0.669733,0.214815,0.166667,0.750000,0.933013,0.0,0.0,0.698409,0.205556,0.166667,0.804381,0.896677,0.0,0.0,0.672763,0.198148,0.166667,0.853553,0.853553,0.0,0.0,0.696678,0.190741,0.166667,0.896677,0.804381,0.0,0.0,0.659777,0.183333,0.166667,0.933013,0.750000,0.0,0.0,0.683476,0.175926,0.166667,0.961940,0.691342,...,0.166667,0.370590,0.982963,0.0,0.0,0.494752,0.272222,0.166667,0.434737,0.995722,0.0,0.0,0.512282,0.274074,0.166667,0.500000,1.000000,0.0,0.0,0.507845,0.275926,0.166667,0.565263,0.995722,0.0,0.0,0.532626,0.277778,0.166667,0.629410,0.982963,0.0,0.0,0.522779,0.281481,0.166667,0.691342,0.961940,0.0,0

In [98]:
testset9 = testset7.iloc[-48:,:] #season을 다시 붙인다면 testset7->8 근데 여기 이해가 안돼.. 48 뭐임? 하루치?
testset9

,RH(t-144),T(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHI(t-144),TARGET(t-144),RH(t-143),T(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHI(t-143),TARGET(t-143),RH(t-142),T(t-142),SH(t-142),sin_time(t-142),cos_time(t-142),GHI(t-142),TARGET(t-142),RH(t-141),T(t-141),SH(t-141),sin_time(t-141),cos_time(t-141),GHI(t-141),TARGET(t-141),RH(t-140),T(t-140),SH(t-140),sin_time(t-140),cos_time(t-140),GHI(t-140),TARGET(t-140),RH(t-139),T(t-139),SH(t-139),sin_time(t-139),cos_time(t-139),...,SH(t-6),sin_time(t-6),cos_time(t-6),GHI(t-6),TARGET(t-6),RH(t-5),T(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1)
288,0.512282,0.274074,0.166667,0.500000,1.000000,0.000000,0.000000,0.507845,0.275926,0.166667,0.565263,0.995722,0.000000,0.000000,0.532626,0.277778,0.166667,0.629410,0.982963,0.000000,0.000000,0.522779,0.281481,0.166667,0.691342,0.961940,0.000000,0.000000,0.550698,0.283333,0.166667,0.750000,0.933013,0.000000,0.000000,0.546045,0.285185,0.166667,0.804381,0.896677,...,0.166667,0.146447,0.853553,0.000000,0.000000,0.547343,0.140741,0.166667,0.195619,0.896677,0.000000,0.000000,0.536306,0.142593,0.166667,0.250000,0.933013,0.000000,0.000000,0.536306,0.142593,0.166667,0.308658,0.961940,0.000000,0.000000,0.521805,0.144444,0.166667,0.370590,0.982963,0.000000,0.000000,0.512174,0.148148,0.166667,0.434737,0.995722,0.000000,0.000000
289,0.507845,0.275926,0.166667,0.565263,0.995722,0.000000,0.000000,0.532626,0.277778,0.166667,0.629410,0.982963,0.000000,0.000000,0.522779,0.281481,0.166667,0.691342,0.961940,0.000000,0.000000,0.550698,0.283333,0.166667,0.750000,0.933013,0.000000,0.000000,0.546045,0.285185,0.166667,0.804381,0.896677,0.000000,0.000000,0.574613,0.285185,0.166667,0.853553,0.853553,...,0.166667,0.195619,0.896677,0.000000,0.000000,0.536306,0.142593,0.166667,0.250000,0.933013,0.000000,0.000000,0.536306,0.142593,0.166667,0.308658,0.961940,0.000000,0.000000,0.521805,0.144444,0.166667,0.370590,0.982963,0.000000,0.000000,0.512174,0.148148,0.166667,0.434737,0.995722,0.000000,0.000000,0.484904,0.153704,0.166667,0.500000,1.000000,0.000000,0.000000
290,0.532626,0.277778,0.166667,0.629410,0.982963,0.000000,0.000000,0.522779,0.281481,0.166667,0.691342,0.961940,0.000000,0.000000,0.550698,0.283333,0.166667,0.750000,0.933013,0.000000,0.000000,0.546045,0.285185,0.166667,0.804381,0.896677,0.000000,0.000000,0.574613,0.285185,0.166667,0.853553,0.853553,0.000000,0.000000,0.569744,0.287037,0.166667,0.896677,0.804381,...,0.166667,0.250000,0.933013,0.000000,0.000000,0.536306,0.142593,0.166667,0.308658,0.961940,0.000000,0.000000,0.521805,0.144444,0.166667,0.370590,0.982963,0.000000,0.000000,0.512174,0.148148,0.166667,0.434737,0.995722,0.000000,0.000000,0.484904,0.153704,0.166667,0.500000,1.000000,0.000000,0.000000,0.476031,0.157407,0.166667,0.565263,0.995722,0.000000,0.000000
291,0.522779,0.281481,0.166667,0.691342,0.961940,0.000000,0.000000,0.550698,0.283333,0.166667,0.750000,0.933013,0.000000,0.000000,0.546045,0.285185,0.166667,0.804381,0.896677,0.000000,0.000000,0.574613,0.285185,0.166667,0.853553,0.853553,0.000000,0.000000,0.569744,0.287037,0.166667,0.896677,0.804381,0.000000,0.000000,0.585002,0.287037,0.166667,0.933013,0.750000,...,0.166667,0.308658,0.961940,0.000000,0.000000,0.521805,0.144444,0.166667,0.370590,0.982963,0.000000,0.000000,0.512174,0.148148,0.166667,0.434737,0.995722,0.000000,0.000000,0.484904,0.153704,0.166667,0.500000,1.000000,0.000000,0.000000,0.476031,0.157407,0.166667,0.565263,0.995722,0.000000,0.000000,0.452116,0.161111,0.166667,0.629410,0.982963,0.000000,0.000000
292,0.550698,0.283333,0.166667,0.750000,0.933013,0.000000,0.000000,0.546045,0.285185,0.166667,0.804381,0.896677,0.000000,0.000000,0.574613,0.285185,0.166667,0.853553,0.853553,

In [99]:
df_test = []

# 예시 코드
# 아직 유진이 어떻게 변수 추가할 지 몰라서 to_supervised는 쓰지 않았음.

for i in range(81):
    file_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/' + str(i) + '.csv'
    new_path = '/content/drive/MyDrive/Jupyter/unlimited_power/raw_data/test/adj_test/' + str(i) + '.csv'
    file_name = str(i) + '.csv'

    
    # adj_test에서 가져와야 하므로 new_path에서 불러온다
    temp = pd.read_csv(new_path)
    temp.drop('Unnamed: 0', axis = 1, inplace = True)
    testset = temp.copy()

    # 24로 저장한 24절기 season으로 이름 바꾸기
    testset.rename(columns = {'24' : 'season'}, inplace = True)

    # 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
    testset2 = drop_clms(testset)
    testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
    testset3 = cos_time(testset2)
    testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]

    testset5 = testset4.copy()
    testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)
    testset6 = testset5[['RH', 'T', 'SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']] #'DHI', 'DNI', 'season'
    testset6 = pd.DataFrame(scaler.transform(testset6), columns=testset6.columns)
    testset7 = test_to_supervised(testset6, n_days)

#    testset8 = encoding(testset7)
#    testset8.drop('season', axis = 1, inplace = True)
    testset9 = testset7.iloc[-48:,:]

    df_test.append(testset9)

X_test = pd.concat(df_test)
# X_test = X_test.iloc[:, :n_obs]
X_test 

,RH(t-144),T(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHI(t-144),TARGET(t-144),RH(t-143),T(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHI(t-143),TARGET(t-143),RH(t-142),T(t-142),SH(t-142),sin_time(t-142),cos_time(t-142),GHI(t-142),TARGET(t-142),RH(t-141),T(t-141),SH(t-141),sin_time(t-141),cos_time(t-141),GHI(t-141),TARGET(t-141),RH(t-140),T(t-140),SH(t-140),sin_time(t-140),cos_time(t-140),GHI(t-140),TARGET(t-140),RH(t-139),T(t-139),SH(t-139),sin_time(t-139),cos_time(t-139),...,SH(t-6),sin_time(t-6),cos_time(t-6),GHI(t-6),TARGET(t-6),RH(t-5),T(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1)
288,0.557299,0.353704,0.166667,0.500000,1.000000,0.0,0.0,0.580132,0.344444,0.166667,0.565263,0.995722,0.0,0.0,0.629477,0.337037,0.166667,0.629410,0.982963,0.0,0.0,0.634672,0.335185,0.166667,0.691342,0.961940,0.0,0.0,0.672113,0.333333,0.166667,0.750000,0.933013,0.0,0.0,0.672113,0.333333,0.166667,0.804381,0.896677,...,0.166667,0.146447,0.853553,0.000000,0.000000,0.813224,0.303704,0.166667,0.195619,0.896677,0.000000,0.000000,0.805216,0.303704,0.166667,0.250000,0.933013,0.000000,0.000000,0.805216,0.303704,0.166667,0.308658,0.961940,0.0,0.0,0.798507,0.303704,0.166667,0.370590,0.982963,0.0,0.0,0.805108,0.301852,0.166667,0.434737,0.995722,0.0,0.0
289,0.580132,0.344444,0.166667,0.565263,0.995722,0.0,0.0,0.629477,0.337037,0.166667,0.629410,0.982963,0.0,0.0,0.634672,0.335185,0.166667,0.691342,0.961940,0.0,0.0,0.672113,0.333333,0.166667,0.750000,0.933013,0.0,0.0,0.672113,0.333333,0.166667,0.804381,0.896677,0.0,0.0,0.655557,0.333333,0.166667,0.853553,0.853553,...,0.166667,0.195619,0.896677,0.000000,0.000000,0.805216,0.303704,0.166667,0.250000,0.933013,0.000000,0.000000,0.805216,0.303704,0.166667,0.308658,0.961940,0.000000,0.000000,0.798507,0.303704,0.166667,0.370590,0.982963,0.0,0.0,0.805108,0.301852,0.166667,0.434737,0.995722,0.0,0.0,0.793529,0.300000,0.166667,0.500000,1.000000,0.0,0.0
290,0.629477,0.337037,0.166667,0.629410,0.982963,0.0,0.0,0.634672,0.335185,0.166667,0.691342,0.961940,0.0,0.0,0.672113,0.333333,0.166667,0.750000,0.933013,0.0,0.0,0.672113,0.333333,0.166667,0.804381,0.896677,0.0,0.0,0.655557,0.333333,0.166667,0.853553,0.853553,0.0,0.0,0.650254,0.335185,0.166667,0.896677,0.804381,...,0.166667,0.250000,0.933013,0.000000,0.000000,0.805216,0.303704,0.166667,0.308658,0.961940,0.000000,0.000000,0.798507,0.303704,0.166667,0.370590,0.982963,0.000000,0.000000,0.805108,0.301852,0.166667,0.434737,0.995722,0.0,0.0,0.793529,0.300000,0.166667,0.500000,1.000000,0.0,0.0,0.800130,0.298148,0.166667,0.565263,0.995722,0.0,0.0
291,0.634672,0.335185,0.166667,0.691342,0.961940,0.0,0.0,0.672113,0.333333,0.166667,0.750000,0.933013,0.0,0.0,0.672113,0.333333,0.166667,0.804381,0.896677,0.0,0.0,0.655557,0.333333,0.166667,0.853553,0.853553,0.0,0.0,0.650254,0.335185,0.166667,0.896677,0.804381,0.0,0.0,0.613678,0.338889,0.166667,0.933013,0.750000,...,0.166667,0.308658,0.961940,0.000000,0.000000,0.798507,0.303704,0.166667,0.370590,0.982963,0.000000,0.000000,0.805108,0.301852,0.166667,0.434737,0.995722,0.000000,0.000000,0.793529,0.300000,0.166667,0.500000,1.000000,0.0,0.0,0.800130,0.298148,0.166667,0.565263,0.995722,0.0,0.0,0.782599,0.296296,0.166667,0.629410,0.982963,0.0,0.0
292,0.672113,0.333333,0.166667,0.750000,0.933013,0.0,0.0,0.672113,0.333333,0.166667,0.804381,0.896677,0.0,0.0,0.655557,0.333333,0.166667,0.853553,0.853553,0.0,0.0,0.650254,0.335185,0.166667,0.896677,0.804381,0.0,0.0,0.613678,0.338889,0.166667,0.933013,0.750000,0.0,0.0,0.609458,0.340741,0.166667,0.961940,0.691342,...,0.166667,0.370590,0.982963,0.000000,0.000000,0.805108,0.301852,0.166667,0.434737,0.995722,0.000000,0.000000,0.793529,0.300000,0.166667,0.500000,1.000000,0.000000,0.000000,0.8001

In [100]:
X_train_1, X_valid_1, Y_train_1,  Y_valid_1 = train_test_split(X, y_1, test_size=0.30, random_state=42)
X_train_2, X_valid_2, Y_train_2,  Y_valid_2 = train_test_split(X, y_2, test_size=0.30, random_state=42)

qq = 0
quantiles = [0.1-qq, 0.2-qq, 0.3-qq, 0.4-qq, 0.5, 0.6-qq, 0.7-qq, 0.8-qq, 0.9-qq]

In [101]:
# LGBM 모델을 구축하고 예측까지 하는 함수를 만든다. 
def CAT(q, X_train, Y_train, X_valid, Y_valid, X_test):
    
    # (a) 모델링
    model = CatBoostRegressor(loss_function='Quantile:alpha=%f'%q, iterations=20000, random_seed=42, learning_rate=0.1, task_type='GPU', use_best_model=True)          
        
    model.fit(X_train, Y_train, eval_set=[(X_valid, Y_valid)], early_stopping_rounds=500, verbose=500)


        # verbose : eval metric이 이 숫자만큼의 round가 지난 다음 자동으로 출력된다.
        # early_stopping_rounds : 이 숫자가 가기 전까지 validation score가 증가하지 않으면 round를 멈춘다. 

    # (b) 예측
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

In [102]:
# Target 예측

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):

    CAT_models=[]
    CAT_actual_pred = pd.DataFrame()

    for q in quantiles:
        print(q)
        pred, model = CAT(q, X_train, Y_train, X_valid, Y_valid, X_test)
        CAT_models.append(model)
        CAT_actual_pred = pd.concat([CAT_actual_pred,pred],axis=1)

    CAT_actual_pred.columns=quantiles
    
    return CAT_models, CAT_actual_pred

In [103]:
tick = time.time()

# Target1

models_1, results_1 = train_data(X_train_1, Y_train_1, X_valid_1, Y_valid_1, X_test)
results_1.sort_index()[:48]

0.1
0:	learn: 1.7804601	test: 1.8057808	best: 1.8057808 (0)	total: 21.2ms	remaining: 7m 3s
500:	learn: 1.7312469	test: 1.7567247	best: 1.7567247 (500)	total: 9.59s	remaining: 6m 13s
1000:	learn: 1.6962888	test: 1.7223694	best: 1.7223656 (999)	total: 18.9s	remaining: 5m 58s
1500:	learn: 1.6669501	test: 1.6931666	best: 1.6931666 (1500)	total: 28.3s	remaining: 5m 48s
2000:	learn: 1.6430120	test: 1.6688943	best: 1.6688943 (2000)	total: 37.6s	remaining: 5m 37s
2500:	learn: 1.6185009	test: 1.6441295	best: 1.6441275 (2499)	total: 46.9s	remaining: 5m 28s
3000:	learn: 1.5994981	test: 1.6251274	best: 1.6251274 (3000)	total: 56.1s	remaining: 5m 18s
3500:	learn: 1.5828842	test: 1.6087118	best: 1.6087070 (3499)	total: 1m 5s	remaining: 5m 8s
4000:	learn: 1.5694232	test: 1.5957590	best: 1.5957590 (4000)	total: 1m 14s	remaining: 4m 58s
4500:	learn: 1.5569704	test: 1.5834411	best: 1.5834411 (4500)	total: 1m 23s	remaining: 4m 49s
5000:	learn: 1.5467598	test: 1.5734020	best: 1.5734002 (4999)	total: 1m 33

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,-0.06,0.01,-0.11,-0.15,-0.40,-0.08,-0.59,2.17,5.34
1,-0.13,0.03,-0.10,-0.01,-0.36,0.07,-0.46,2.11,5.27
2,-0.17,0.01,-0.09,-0.17,0.40,0.17,-0.70,2.61,5.51
3,-0.16,0.04,-0.04,-0.10,0.77,0.54,-0.65,2.03,4.98
4,-0.15,0.05,0.02,0.16,1.09,0.49,-0.57,2.01,5.12
5,-0.08,0.11,-0.00,0.31,1.47,0.33,-0.58,1.57,4.67
6,-0.10,-0.03,-0.10,0.28,1.65,-0.15,-0.16,1.37,4.42
7,-0.10,0.05,-0.21,-0.06,2.38,0.03,-1.01,0.69,3.99
8,-0.12,-0.06,-0.39,-0.18,2.18,0.04,-0.55,0.65,3.79
9,-0.12,-0.26,-0.44,-0.58,2.32,-0.36,-0.46,2.35,5.06


In [104]:
# Target2

models_2, results_2 = train_data(X_train_2, Y_train_2, X_valid_2, Y_valid_2, X_test)
tock = time.time()
results_2.sort_index()

0.1
0:	learn: 1.7822286	test: 1.8028445	best: 1.8028445 (0)	total: 19.3ms	remaining: 6m 25s
500:	learn: 1.7371182	test: 1.7582040	best: 1.7582040 (500)	total: 9.21s	remaining: 5m 58s
1000:	learn: 1.7056602	test: 1.7273203	best: 1.7273203 (1000)	total: 18.4s	remaining: 5m 49s
1500:	learn: 1.6774218	test: 1.6990835	best: 1.6990835 (1500)	total: 27.8s	remaining: 5m 42s
2000:	learn: 1.6555603	test: 1.6773054	best: 1.6773054 (2000)	total: 37.7s	remaining: 5m 39s
2500:	learn: 1.6335365	test: 1.6551698	best: 1.6551698 (2500)	total: 46.9s	remaining: 5m 27s
3000:	learn: 1.6144678	test: 1.6363977	best: 1.6363977 (3000)	total: 56.1s	remaining: 5m 17s
3500:	learn: 1.5994032	test: 1.6212885	best: 1.6212885 (3500)	total: 1m 5s	remaining: 5m 8s
4000:	learn: 1.5872077	test: 1.6091773	best: 1.6091773 (4000)	total: 1m 14s	remaining: 4m 57s
4500:	learn: 1.5744396	test: 1.5963000	best: 1.5963000 (4500)	total: 1m 23s	remaining: 4m 48s
5000:	learn: 1.5633397	test: 1.5854084	best: 1.5854084 (5000)	total: 1m 

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,-0.04,0.21,-0.14,-0.07,-0.92,0.03,-0.70,2.31,5.77
1,-0.01,0.19,-0.21,0.02,-0.24,-0.00,-0.38,2.46,6.16
2,-0.12,0.13,-0.26,0.17,0.06,-0.05,-0.48,2.44,6.04
3,-0.06,0.11,-0.29,0.05,0.23,0.24,-0.65,2.03,6.15
4,-0.04,0.11,-0.17,0.15,0.87,0.16,-0.47,1.80,6.07
...,...,...,...,...,...,...,...,...,...
3883,-0.05,0.02,0.24,0.06,0.40,0.60,-0.65,-1.73,1.12
3884,-0.02,-0.02,0.21,0.18,0.25,0.06,-0.22,-1.61,0.11
3885,-0.03,0.05,0.17,0.10,0.32,-0.20,-0.89,-1.63,-0.33
3886,-0.00,0.13,0.15,0.07,-0.20,0.43,-0.55,-2.78,-1.33


In [105]:
print('소요시간 :', tock-tick)

print(results_1.shape, results_2.shape)

소요시간 : 6532.296728849411
(3888, 9) (3888, 9)


In [106]:
# 출력

submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.sort_index().values
submission2 = submission.set_index('id')
submission3 = not_minus(submission2)

In [107]:
submission3.to_csv('submission_cat.csv')
!cp submission_cat.csv "drive/My Drive/"